In [1]:
import pyodbc
import pandas as pd
import numpy as np
from scipy import optimize
from scipy.optimize import minimize
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
class DBConnector:
    def __init__(self, server_name, username, password, db_name, connection_str):
        self.sever_name = server_name
        self.username = username
        self.password = password
        self.db_name = db_name
        self.connection_str = connection_str
        self.connection = pyodbc.connect(self.connection_str)

    def test_connection(self):
        try:
            db = self.connection
            cursor = db.cursor()
            cursor.execute("SELECT @@VERSION")
            results = cursor.fetchone()
            # Check if anything at all is returned
            if results:
                return True
            else:
                return False
        except pyodbc.Error as ex:
            print(ex)
            print("Error in connection!")
            sqlstate = ex.args[0]
            if sqlstate == '28000':
                print("LDAP Connection failed: check password")
        return False

    def query(self, query_str):
        """This function returns a dataframe of result list"""
        return pd.read_sql_query(query_str, self.connection)

    def all_columns_name(self, table_name):
        """This function returns a dataframe of result list"""
        query_str = f"select COLUMN_NAME from INFORMATION_SCHEMA.COLUMNS where TABLE_NAME='{table_name}'"
        print("All columns names: ", query_str)
        return pd.read_sql_query(query_str, self.connection)

driver="SQL Server"
servername = 'QUOC-CUONG'
username = 'sa'
password = 'cuong300599'
db_name = 'OnlinePhoneShop'

str_for_connection = "Driver={driver};Server={servername};UID={username};PWD={password};Database={db_name};"\
    .format(driver=driver, servername=servername, username=username, password=password, db_name=db_name)

connector = DBConnector(servername, username, password, db_name, str_for_connection)

In [3]:
query_str = "select distinct u.user_id, p.product_id, dbo.fn_get_rating(u.user_id, p.product_id) as score from users u, products p"
df_users_products_ratings = connector.query(query_str)
df_users_products_ratings_pivotted = df_users_products_ratings.pivot(index='product_id', columns='user_id', values='score').fillna(0)
print("Number of products (rows): ", df_users_products_ratings_pivotted.shape[0])
print("Number of  users (columns): ", df_users_products_ratings_pivotted.shape[1])

# Save to csv file
df_users_products_ratings_pivotted.to_csv(r'D:\PhoneShopML\data\rating_matrix.csv', header=True, index=True, encoding='utf-8-sig')

df_users_products_ratings_pivotted.head(20)

Number of products (rows):  63
Number of  users (columns):  77


user_id,US041020210001,US061020210001,US071020210001,US191020210001,US191020210002,US191020210003,US191020210004,US191020210005,US191020210006,US191020210007,US191020210008,US191020210009,US191020210010,US191020210011,US191020210012,US251020210001,US251020210013,US251020210014,US251020210015,US251020210016,US251020210017,US251020210018,US251020210019,US251020210020,US271020210001,US271020210021,US271020210022,US271020210023,US271020210024,US271020210025,US271020210026,US271020210027,US271020210028,US271020210029,US271020210030,US271020210031,US271020210032,US271020210033,US271020210034,US271020210035,US271020210036,US271020210037,US281020210001,US281020210038,US281020210039,US281020210040,US281020210041,US281020210042,US281020210043,US281020210044,US281020210045,US281020210046,US281020210047,US281020210048,US281020210049,US281020210050,US281020210051,US281020210052,US281020210053,US281020210054,US281020210055,US281020210056,US281020210057,US281020210058,US281020210059,US281020210060,US281020210061,US281020210062,US281020210063,US281020210064,US281020210065,US281020210066,US281020210067,US281020210068,US281020210069,US281020210070,US281020210071
product_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
PD041020210001,4.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,0.0
PD041020210002,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.5,0.0,0.0,0.0,2.0,2.5,0.0,3.0,3.0,0.0,4.0,0.0,4.0,4.5,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0
PD041020210003,4.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,5.0
PD041020210004,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,4.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,3.0,0.0,0.0,4.5,0.0,4.5,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0
PD041020210005,0.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,2.5,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,4.5,4.5,0.0,0.0,0.0,0.0,0.0
PD081020210006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
PD081020210007,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,1.5,0.0,0.0,0.0,1.5,1.5,0.0,0.0,1.5,0.0,0.0,0.0,1.0,2.0,2.5,2.0,0.0,0.0,2.0,0.0,3.0,3.0,0.0,0.0,0.0,4.5,4.5,0.0,4.0,0.0,5.0,0.0,0.0,0.0,0.0
PD111120210001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PD111120210057,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [4]:
df_is_rated = df_users_products_ratings_pivotted.copy()
# df_is_rated.head(20)

column_names = df_users_products_ratings_pivotted.columns.values
print(column_names)

for name in column_names:
    df_is_rated.loc[df_is_rated[name] > 0, name] = 1

df_is_rated.head(20)

['US041020210001' 'US061020210001' 'US071020210001' 'US191020210001'
 'US191020210002' 'US191020210003' 'US191020210004' 'US191020210005'
 'US191020210006' 'US191020210007' 'US191020210008' 'US191020210009'
 'US191020210010' 'US191020210011' 'US191020210012' 'US251020210001'
 'US251020210013' 'US251020210014' 'US251020210015' 'US251020210016'
 'US251020210017' 'US251020210018' 'US251020210019' 'US251020210020'
 'US271020210001' 'US271020210021' 'US271020210022' 'US271020210023'
 'US271020210024' 'US271020210025' 'US271020210026' 'US271020210027'
 'US271020210028' 'US271020210029' 'US271020210030' 'US271020210031'
 'US271020210032' 'US271020210033' 'US271020210034' 'US271020210035'
 'US271020210036' 'US271020210037' 'US281020210001' 'US281020210038'
 'US281020210039' 'US281020210040' 'US281020210041' 'US281020210042'
 'US281020210043' 'US281020210044' 'US281020210045' 'US281020210046'
 'US281020210047' 'US281020210048' 'US281020210049' 'US281020210050'
 'US281020210051' 'US281020210052'

user_id,US041020210001,US061020210001,US071020210001,US191020210001,US191020210002,US191020210003,US191020210004,US191020210005,US191020210006,US191020210007,US191020210008,US191020210009,US191020210010,US191020210011,US191020210012,US251020210001,US251020210013,US251020210014,US251020210015,US251020210016,US251020210017,US251020210018,US251020210019,US251020210020,US271020210001,US271020210021,US271020210022,US271020210023,US271020210024,US271020210025,US271020210026,US271020210027,US271020210028,US271020210029,US271020210030,US271020210031,US271020210032,US271020210033,US271020210034,US271020210035,US271020210036,US271020210037,US281020210001,US281020210038,US281020210039,US281020210040,US281020210041,US281020210042,US281020210043,US281020210044,US281020210045,US281020210046,US281020210047,US281020210048,US281020210049,US281020210050,US281020210051,US281020210052,US281020210053,US281020210054,US281020210055,US281020210056,US281020210057,US281020210058,US281020210059,US281020210060,US281020210061,US281020210062,US281020210063,US281020210064,US281020210065,US281020210066,US281020210067,US281020210068,US281020210069,US281020210070,US281020210071
product_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
PD041020210001,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
PD041020210002,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
PD041020210003,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
PD041020210004,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
PD041020210005,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
PD081020210006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
PD081020210007,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
PD111120210001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PD111120210057,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [5]:
import numpy as np

def cofiCostFunc(params, Y, R, num_users, num_products, num_features, lambda_var):
    # Unfold the U and W matrices from params
    X = np.reshape(params[:num_products*num_features], (num_products, num_features), order='F')
    Theta = np.reshape(params[num_products*num_features:], (num_users, num_features), order='F')

    # You need to return the following values correctly
    J = 0
    X_grad = np.zeros(X.shape)
    Theta_grad = np.zeros(Theta.shape)

    squared_error = np.power(np.dot(X,Theta.T) - Y,2)

    # for cost function, sum only i,j for which R(i,j)=1
    J = (1/2.) * np.sum(squared_error * R)

    X_grad = np.dot(( np.dot(X, Theta.T) - Y ) * R, Theta)
    Theta_grad = np.dot((( np.dot(X, Theta.T) - Y ) * R).T, X)

    ### COST FUNCTION WITH REGULARIZATION
    # only add regularized cost to J now
    J = J + (lambda_var/2.)*( np.sum( np.power(Theta, 2) ) + np.sum( np.power(X, 2) ) )

    ### GRADIENTS WITH REGULARIZATION
    # only add regularization terms
    X_grad = X_grad + lambda_var*X
    Theta_grad = Theta_grad + lambda_var*Theta

    # =============================================================

    grad = np.concatenate((X_grad.reshape(X_grad.size, order='F'), Theta_grad.reshape(Theta_grad.size, order='F')))

    return J, grad

In [6]:
def normalizeRatings(Y, R):
    #NORMALIZERATINGS Preprocess data by subtracting mean rating for every 
    #movie (every row)
    #   [Ynorm, Ymean] = NORMALIZERATINGS(Y, R) normalized Y so that each movie
    #   has a rating of 0 on average, and returns the mean rating in Ymean.
    #

    m, n = Y.shape
    Ymean = np.zeros((m, 1))
    Ynorm = np.zeros(Y.shape)
    for i in range(m):
        idx = R[i, :] == 1
        Ymean[i] = np.mean(Y[i, idx])
        Ynorm[i, idx] = Y[i, idx] - Ymean[i]

    return Ynorm, Ymean

In [7]:
Y = df_users_products_ratings_pivotted.to_numpy()
R = df_is_rated.to_numpy()

print("Rating numpy array: \n", Y)
print("Rating numpy array's shape: ", Y.shape)
print("Check rated numpy array\n", R)
print("Check rated numpy array's shape: \n", R.shape)

Rating numpy array: 
 [[4.  0.  2.5 ... 5.  5.  0. ]
 [2.5 0.  0.  ... 0.  0.  0. ]
 [4.  0.  0.  ... 5.  0.  5. ]
 ...
 [0.  0.  0.  ... 0.  5.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]]
Rating numpy array's shape:  (63, 77)
Check rated numpy array
 [[1. 0. 1. ... 1. 1. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 1. 0. 1.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Check rated numpy array's shape: 
 (63, 77)


In [8]:
#  Normalize Ratings
[Ynorm, Ymean] = normalizeRatings(Y, R)

#  Useful Values
num_users = Y.shape[1]
num_products = Y.shape[0]
num_features = 3

# Set Initial Parameters (Theta, X)
X = np.random.randn(num_products, num_features)
Theta = np.random.randn(num_users, num_features)
# X = np.random.standard_normal(num_products * num_features) 
# Theta = np.random.standard_normal(num_users * num_features)

initial_parameters = np.concatenate((X.reshape(X.size, order='F'), Theta.reshape(Theta.size, order='F')))

# Set options
maxiter = 100
options = {'disp': True, 'maxiter':maxiter}
lambda_var=3

# Create "short hand" for the cost function to be minimized
def costFunc(initial_parameters):
    return cofiCostFunc(initial_parameters, Y, R, num_users, num_products, num_features, lambda_var)

# Set Regularization
results = minimize(costFunc, x0=initial_parameters, options=options, method="L-BFGS-B", jac=True)
theta = results["x"]

# Unfold the returned theta back into U and W
X = np.reshape(theta[:num_products*num_features], (num_products, num_features), order='F')
Theta = np.reshape(theta[num_products*num_features:], (num_users, num_features), order='F')

c:\users\quoc cuong\appdata\local\programs\python\python39\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\users\quoc cuong\appdata\local\programs\python\python39\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [9]:
# Make prediction
# p = X.dot(Theta.T)
p = np.dot(X, Theta.T)
print("Shape of prediction array: ", p.shape)
print("Prediction Array: \n", p)

Shape of prediction array:  (63, 77)
Prediction Array: 
 [[2.72437409 2.45700831 1.35846963 ... 4.27960115 4.4816773  4.63081144]
 [2.75370465 2.7541791  1.38573046 ... 4.37031428 4.61957725 4.89119578]
 [2.92699467 2.61051752 1.45812798 ... 4.59303277 4.80530291 4.95259548]
 ...
 [2.39790601 2.20462612 1.19755518 ... 3.77341051 3.95845189 4.10939998]
 [2.18479269 2.0210692  1.09151658 ... 3.43948878 3.61053726 3.75549229]
 [1.28831794 1.2428204  0.64635295 ... 2.03767868 2.14632079 2.25118735]]


In [10]:
def get_product_info(product_id):
    query_str = f"select product_name, unit_price, label from products p where product_id='{product_id}'"
    df_name= connector.query(query_str)
    product_name, unit_price, label = df_name.values.tolist()[0]

    return product_name, unit_price, label

product_name, unit_price, label = get_product_info('PD271020210019')
print("Name: ", product_name)
print("Price: ", unit_price)
print("Label: ", label)

Name:  Điện thoại OPPO Reno6 Pro 5G
Price:  18490000.0
Label:  1


In [11]:
label2type={0:"Phụ kiện", 1: "Gaming/ Cấu hình cao", 2:"Giải trí thông thường", 3:"Chỉ nghe và gọi"}

In [12]:
# Get the prediction of rating for the user
# order start from 0
import math
# US281020210062
user_id = 'US281020210062'
user_index_in_matrix = df_users_products_ratings_pivotted.columns.get_loc(user_id)
print("Index of user in matrix: ", user_index_in_matrix)

print("Y mean flatten: ", Ymean.flatten().shape)
print("P shape: ", p[:,user_index_in_matrix].shape)
user_prediction = p[:,user_index_in_matrix] + Ymean.flatten()
user_prediction = [x for x in user_prediction if str(x) != 'nan']

# Sort user prediction with descending order of score
sorted_indices = np.argsort(user_prediction)[::-1]
list_products = df_users_products_ratings_pivotted.index.values.tolist() 
n_top = 10
print("Collaborative Filtering recommendation products:\n")
cols = ['index', 'product_id', 'product_name', 'unit_price', 'dedicated', 'predicted_score']
recommended_products = []
for position in range(n_top):
    index = sorted_indices[position]
#     print(f"Product at Index: {index} - predict score: {user_prediction[index]} \nRecommend product: {get_product_name(list_products[index])}\n")
    product_name, unit_price, label = get_product_info(list_products[index])
    recommended_products.append([index, list_products[index], product_name, unit_price, label, user_prediction[index]])

df_recommended_products = pd.DataFrame(recommended_products, columns=cols)
df_recommended_products["dedicated"] = df_recommended_products["dedicated"].map(label2type)
df_recommended_products.head(n_top)

Index of user in matrix:  67
Y mean flatten:  (63,)
P shape:  (63,)
Collaborative Filtering recommendation products:



,index,product_id,product_name,unit_price,dedicated,predicted_score
0,7,PD111120210001,Điện Thoại OnePlus Nord CE 5G (12GB/256G) - Hà...,7990000.0,Gaming/ Cấu hình cao,7.626530
1,24,PD271020210018,Điện Thoại Xiaomi Mi 11 Lite 5G NE 6GB l 128GB...,9400000.0,Gaming/ Cấu hình cao,7.601436
2,23,PD271020210017,Điện Thoại Oppo Reno 6 5G (8GB/128G) - Hàng Ch...,12990000.0,Gaming/ Cấu hình cao,7.585585
3,25,PD271020210019,Điện thoại OPPO Reno6 Pro 5G,18490000.0,Gaming/ Cấu hình cao,7.543983
4,4,PD041020210005,Vivo y12,10000000.0,Giải trí thông thường,7.450769
5,9,PD151020210001,Điện Thoại iPhone 12 Pro Max 128GB - Hàng Chín...,30990000.0,Gaming/ Cấu hình cao,7.381627
6,2,PD041020210003,Điện Thoại Vsmart Joy 4 - Hàng Chính Hãng,3059000.0,Giải trí thông thường,7.204948
7,15,PD191020210011,Ốp lưng nhám mờ cho Samsung Galaxy A12 chống s...,40000.0,Phụ kiện,6.966199
8,28,PD271020210022,Điện Thoại Samsung Galaxy S21 Ultra 5G (12GB/1...,25990000.0,Gaming/ Cấu hình cao,6.879738
9,21,PD271020210015,Điện thoại Xiaomi 11T 5G 256GB,11990000.0,Gaming/ Cấu hình cao,6.867280


In [13]:
print("User's ratings in the past:\n")
query_str = f"select p.product_id, p.product_name, label from products p where p.product_id in (select product_id from ratings where user_id='{user_id}')"
df_past_ratings = connector.query(query_str)
df_past_ratings["dedicated"] = df_past_ratings["label"].map(label2type)

df_past_ratings.head(20)

User's ratings in the past:



,product_id,product_name,label,dedicated
0,PD041020210002,Tai nghe Bluetooth AirPods Pro Wireless Charge...,0,Phụ kiện
1,PD081020210007,Máy Tính Bảng HUAWEI MatePad T 10 | Hiệu Suất ...,1,Gaming/ Cấu hình cao
2,PD151020210001,Điện Thoại iPhone 12 Pro Max 128GB - Hàng Chín...,1,Gaming/ Cấu hình cao
3,PD161020210008,Tai Nghe Nhét Tai JBL C150SI - Hàng Chính Hãng,0,Phụ kiện
4,PD271020210041,Điện thoại Realme C25s,2,Giải trí thông thường


# Recommendations for user has not had ratings

In [14]:
# Get list of users haven't rated before
query_str = "select user_id, username from users where user_id not in (select user_id from ratings)"
df_results = connector.query(query_str)

print("Users haven't rated any products: \n")
df_results.head(len(df_results.index))

Users haven't rated any products: 



,user_id,username
0,US251020210019,hientran
1,US251020210017,hoangt
2,US251020210016,khangtran
3,US251020210013,mdang
4,US251020210020,myl
5,US281020210038,mytr
6,US251020210015,namthanh
7,US251020210018,ngoct
8,US251020210014,thienng
9,US191020210011,ynhi


In [15]:
# Get the prediction of rating for the user
# order start from 0
user_id = 'US251020210020'
user_index_in_matrix = df_users_products_ratings_pivotted.columns.get_loc(user_id)
print("Index of user in matrix: ", user_index_in_matrix)

print("Y mean flatten: ", Ymean.flatten().shape)
print("P shape: ", p[:,user_index_in_matrix].shape)
user_prediction = p[:,user_index_in_matrix] + Ymean.flatten()
user_prediction = [x for x in user_prediction if str(x) != 'nan']

# Sort user prediction with descending order of score
sorted_indices = np.argsort(user_prediction)[::-1]
list_products = df_users_products_ratings_pivotted.index.values.tolist() 
n_top = 10
print("Collaborative Filtering recommendation products:\n")
cols = ['index', 'product_id', 'product_name', 'unit_price', 'dedicated', 'predicted_score']
recommended_products = []
for position in range(n_top):
    index = sorted_indices[position]
#     print(f"Product at Index: {index} - predict score: {user_prediction[index]} \nRecommend product: {get_product_name(list_products[index])}\n")
    product_name, unit_price, label = get_product_info(list_products[index])
    recommended_products.append([index, list_products[index], product_name, unit_price, label, user_prediction[index]])

df_recommended_products = pd.DataFrame(recommended_products, columns=cols)
df_recommended_products["dedicated"] = df_recommended_products["dedicated"].map(label2type)
df_recommended_products.head(n_top)

Index of user in matrix:  23
Y mean flatten:  (63,)
P shape:  (63,)
Collaborative Filtering recommendation products:



,index,product_id,product_name,unit_price,dedicated,predicted_score
0,23,PD271020210017,Điện Thoại Oppo Reno 6 5G (8GB/128G) - Hàng Ch...,12990000.0,Gaming/ Cấu hình cao,5.000054
1,21,PD271020210015,Điện thoại Xiaomi 11T 5G 256GB,11990000.0,Gaming/ Cấu hình cao,4.500183
2,25,PD271020210019,Điện thoại OPPO Reno6 Pro 5G,18490000.0,Gaming/ Cấu hình cao,4.500067
3,22,PD271020210016,Điện Thoại Samsung A52S 5G 8GB/128GB - Hàng Ch...,10990000.0,Gaming/ Cấu hình cao,4.500041
4,24,PD271020210018,Điện Thoại Xiaomi Mi 11 Lite 5G NE 6GB l 128GB...,9400000.0,Gaming/ Cấu hình cao,4.333587
5,27,PD271020210021,Điện Thoại iPhone 12 Pro 256GB - Hàng Chính Hãng,27500000.0,Gaming/ Cấu hình cao,4.000275
6,16,PD191020210012,Ốp lưng dẻo trong suốt TPU loại tốt dành cho S...,12000.0,Phụ kiện,4.000263
7,30,PD271020210024,Điện Thoại Samsung Galaxy S21 Plus 5G (8GB/128...,20990000.0,Gaming/ Cấu hình cao,4.000177
8,31,PD271020210025,Điện thoại Vivo X70 Pro 5G (12GB/256GB) - Hàng...,17750000.0,Gaming/ Cấu hình cao,4.000172
9,26,PD271020210020,Điện Thoại iPhone 13 Pro Max 1TB - Hàng Chính ...,47990000.0,Gaming/ Cấu hình cao,3.999986
